In [1]:
!pip install -U langchain_community --quiet
!pip install pypdf --quiet
!pip install instructor --quiet
!pip install faiss-cpu

In [3]:
from configs.maxi_config import API_KEY, ENDPOINT, MODEL, DEPLOYMENT

import os
import torch
import requests
import numpy as np
from tqdm import tqdm
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from embeddings_wrapper import LangchainSentenceTransformer
from typing import Any
from pydantic import BaseModel, ValidationError, Field
from openai import AsyncAzureOpenAI


/Users/maxi/miniforge3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Lee un solo PDF
pdf_path = "datasets/2024/02/8142.pdf"

if not os.path.exists(pdf_path):
    raise FileNotFoundError(f"No se encontró el archivo: {pdf_path}")

# --- Cargar y dividir el PDF ---
print("[INFO] Cargando y dividiendo el PDF...")
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)

try:
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    chunks = splitter.split_documents(documents)
except Exception as e:
    print(f"[ERROR] No se pudo procesar el PDF: {e}")
    chunks = []

print(f"[INFO] Total de fragmentos generados: {len(chunks)}")

# --- Embeddings y FAISS ---
print("[INFO] Generando embeddings...")
embedding_model = LangchainSentenceTransformer()
texts, embeddings = embedding_model.embed_documents(chunks)


[INFO] Cargando y dividiendo el PDF...
[INFO] Total de fragmentos generados: 4
[INFO] Generando embeddings...
[INFO] Filtering 4 chunks...
[INFO] Embedding 4 clean chunks...


Batches: 100%|██████████| 1/1 [00:02<00:00,  2.56s/it]


In [5]:
text_embedding_pairs = list(zip(texts, embeddings))
vectorstore = FAISS.from_embeddings(text_embedding_pairs, embedding_model)
vectorstore.save_local("datasets/fallo_8142_vectorstore")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


## Crear archivo con data de api, y dejarlo afuera del repo, que aca se llame a eso!!!

In [6]:
def create_context(query):
    results = vectorstore.similarity_search(query, k=5)
    # for i, doc in enumerate(results, 1):
    #     print(f"{i}. {doc.page_content.strip()}\n")
    return results

azure_client = AsyncAzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=ENDPOINT,
    api_key=API_KEY
)

async def get_response(query):
    res = await azure_client.chat.completions.create(
        model=DEPLOYMENT,
        messages=[
            {
                "role": "system",
                "content": "Sos un asistente judicial especializado en encontrar semejanzas entre fallos y responder preguntas sobre ellos. Tu tarea es analizar el contexto y responder de manera precisa y concisa.",
            },
            {
                "role": "system", 
                "content": f"El contexto es: {create_context(query)}"
            },
            {
                "role": "user",
                "content": query
            }
        ],
        
    )
    return res

In [7]:
response = await get_response("Que dice el fallo dado?")



In [8]:
respuesta = response.choices[0].message.content
respuesta


'El fallo se refiere a la regulación de honorarios profesionales por los recursos de inaplicabilidad de ley resueltos en dos fechas: 25 de junio de 2020 y 10 de abril de 2023. Se establecen las siguientes sumas a pagar a cada abogado: \n\n1. Para el recurso de 25/6/2020:\n   - Dr. Ricardo Abraham Salvador: $63.900\n   - Dr. Julio Eduardo Amaduzzi: $22.365\n   - Dr. Nicolás Eduardo Bruno: $22.365\n\n2. Para el recurso de 10/4/2023:\n   - Dr. Ricardo Abraham Salvador: $213.000\n   - Dra. María Antonella Arlettaz: $74.550\n   - Dr. Diego G. Spinelli: $74.550\n\nAsimismo, el fallo ha seguido disposiciones legales sobre la notificación y el pago de honorarios, incluyendo detalles sobre aplicación de intereses en caso de mora. Se indica que se notifique conforme a los procedimientos establecidos y se registre adecuadamente.'

In [9]:
from IPython.display import Markdown
display(Markdown(respuesta))


El fallo se refiere a la regulación de honorarios profesionales por los recursos de inaplicabilidad de ley resueltos en dos fechas: 25 de junio de 2020 y 10 de abril de 2023. Se establecen las siguientes sumas a pagar a cada abogado: 

1. Para el recurso de 25/6/2020:
   - Dr. Ricardo Abraham Salvador: $63.900
   - Dr. Julio Eduardo Amaduzzi: $22.365
   - Dr. Nicolás Eduardo Bruno: $22.365

2. Para el recurso de 10/4/2023:
   - Dr. Ricardo Abraham Salvador: $213.000
   - Dra. María Antonella Arlettaz: $74.550
   - Dr. Diego G. Spinelli: $74.550

Asimismo, el fallo ha seguido disposiciones legales sobre la notificación y el pago de honorarios, incluyendo detalles sobre aplicación de intereses en caso de mora. Se indica que se notifique conforme a los procedimientos establecidos y se registre adecuadamente.

## Adaptar esto para que tome los jsons creados por Toni!!!